# Episode IV: Return of the Legs

- Adherence
- Volume, Frequency, Intensity, and Fatigue
- Progression
- Exercise Selection
- Rest Periods
- Tempo
- Periodization

In [56]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
engine = create_engine('sqlite:///../gains.db')

import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme = 'pearl')

import plotly.graph_objs as go
from plotly.offline import plot

In [57]:
def volume(df, routine, breakdown, volume_type):
    '''
    Create pivot table of total volume or relative volume
    per muscle group each week, sorting by the most recent 
    week's volume
    '''
    view = (df
            .loc[(df.routine == routine) & (df.category != 'Cardio')]
            .pivot_table(index = 'date', columns = breakdown, values = volume_type, aggfunc = 'sum')
            .resample('W')
            .sum()
            .astype(int))
    
    return view

In [58]:
sets = pd.read_sql('sets', engine, parse_dates = ['date'])

In [59]:
exercises = pd.read_sql('exercises', engine)

In [60]:
sets = sets.merge(exercises[['exercise', 'variant']], on = 'exercise')

## Adherance

In [61]:
episode = sets[sets.date >= '5-28-2019'].query('rpe >= 6')

In [62]:
grouper = pd.Grouper(key = 'date', freq = 'W')

In [63]:
episode.groupby(grouper).id.nunique().to_frame('Number of Workouts')

,Number of Workouts
date,
2019-06-09,1
2019-06-16,0
2019-06-23,0
2019-06-30,1
2019-07-07,2
2019-07-14,3
2019-07-21,3


## Frequency

In [64]:
episode.pivot_table(index = grouper, columns = 'variant', values = 'id', aggfunc = 'nunique')

variant,pull,push
date,,
2019-06-09,1,1
2019-06-30,1,1
2019-07-07,2,2
2019-07-14,2,2
2019-07-21,2,3


In [65]:
episode.pivot_table(index = grouper, columns = 'category', values = 'id', aggfunc = 'nunique').fillna('')

category,Back,Biceps,Chest,Legs,Shoulders,Triceps
date,,,,,,
2019-06-09,1.0,,1.0,,,
2019-06-30,1.0,1,1.0,,,1
2019-07-07,2.0,1,2.0,,1,1
2019-07-14,2.0,1,2.0,1,1,1
2019-07-21,2.0,1,2.0,1,1,1


## Volume

### Sets

In [66]:
episode.pivot_table(index = grouper, columns = 'variant', values = 'id', aggfunc = 'count')

variant,pull,push
date,,
2019-06-09,1,1
2019-06-30,5,5
2019-07-07,10,12
2019-07-14,10,11
2019-07-21,9,12


In [67]:
episode.pivot_table(index = grouper, columns = 'category', values = 'id', aggfunc = 'count')

category,Back,Biceps,Chest,Legs,Shoulders,Triceps
date,,,,,,
2019-06-09,1.0,NaN,1.0,NaN,NaN,NaN
2019-06-30,5.0,2.0,5.0,NaN,NaN,2.0
2019-07-07,10.0,2.0,12.0,NaN,2.0,1.0
2019-07-14,10.0,2.0,11.0,1.0,2.0,2.0
2019-07-21,9.0,2.0,9.0,2.0,2.0,2.0


### Volume Load

In [68]:
vol = episode.pivot_table(index = grouper, columns = 'variant', values = 'volume', aggfunc = 'sum')

In [69]:
vol

variant,pull,push
date,,
2019-06-09,2400.0,1600.0
2019-06-30,9300.0,8100.0
2019-07-07,16166.0,13365.0
2019-07-14,15442.0,13585.0
2019-07-21,14386.0,13490.0


In [70]:
vol.diff()

variant,pull,push
date,,
2019-06-09,NaN,NaN
2019-06-30,6900.0,6500.0
2019-07-07,6866.0,5265.0
2019-07-14,-724.0,220.0
2019-07-21,-1056.0,-95.0


In [71]:
vol_cat = episode.pivot_table(index = grouper, columns = 'category', values = 'volume', aggfunc = 'sum')

In [72]:
vol_cat

category,Back,Biceps,Chest,Legs,Shoulders,Triceps
date,,,,,,
2019-06-09,2400.0,NaN,1600.0,NaN,NaN,NaN
2019-06-30,9300.0,600.0,8100.0,NaN,NaN,4800.0
2019-07-07,16166.0,600.0,14065.0,NaN,800.0,2508.0
2019-07-14,15442.0,720.0,14085.0,360.0,880.0,5016.0
2019-07-21,14386.0,595.0,12090.0,1000.0,880.0,5016.0


In [73]:
vol_cat.diff()

category,Back,Biceps,Chest,Legs,Shoulders,Triceps
date,,,,,,
2019-06-09,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-30,6900.0,NaN,6500.0,NaN,NaN,NaN
2019-07-07,6866.0,0.0,5965.0,NaN,NaN,-2292.0
2019-07-14,-724.0,120.0,20.0,NaN,80.0,2508.0
2019-07-21,-1056.0,-125.0,-1995.0,640.0,0.0,0.0


### Relative Volume Load

In [74]:
rel_vol = episode.pivot_table(index = grouper, columns = 'variant', values = 'relative_volume', aggfunc = 'sum')

In [75]:
rel_vol

variant,pull,push
date,,
2019-06-09,1655.0,1072.0
2019-06-30,6084.0,5529.0
2019-07-07,11602.0,9819.0
2019-07-14,11948.0,10067.0
2019-07-21,10619.0,9844.0


In [76]:
rel_vol.diff()

variant,pull,push
date,,
2019-06-09,NaN,NaN
2019-06-30,4429.0,4457.0
2019-07-07,5518.0,4290.0
2019-07-14,346.0,248.0
2019-07-21,-1329.0,-223.0


In [77]:
rel_vol_cat = episode.pivot_table(index = grouper, columns = 'category', values = 'relative_volume', aggfunc = 'sum')

In [78]:
rel_vol_cat

category,Back,Biceps,Chest,Legs,Shoulders,Triceps
date,,,,,,
2019-06-09,1655.0,NaN,1072.0,NaN,NaN,NaN
2019-06-30,6084.0,426.0,5529.0,NaN,NaN,3360.0
2019-07-07,11602.0,426.0,10191.0,NaN,632.0,1780.0
2019-07-14,11948.0,510.0,10297.0,284.0,694.0,3560.0
2019-07-21,10619.0,451.0,8721.0,750.0,694.0,3560.0


In [79]:
rel_vol_cat.diff()

category,Back,Biceps,Chest,Legs,Shoulders,Triceps
date,,,,,,
2019-06-09,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-30,4429.0,NaN,4457.0,NaN,NaN,NaN
2019-07-07,5518.0,0.0,4662.0,NaN,NaN,-1580.0
2019-07-14,346.0,84.0,106.0,NaN,62.0,1780.0
2019-07-21,-1329.0,-59.0,-1576.0,466.0,0.0,0.0


## Intensity

In [81]:
episode[episode.stimulus == 'hypertrophy'].pivot_table(index = grouper, columns = ['variant', 'hypertrophy_range'], values = 'id', aggfunc = 'count')

variant              pull             push         
hypertrophy_range [7, 12) [12, 20) [7, 12) [12, 20)
date                                               
2019-06-09            NaN      1.0     1.0      NaN
2019-06-30            3.0      2.0     3.0      2.0
2019-07-07            2.0      2.0     1.0      2.0
2019-07-14            NaN      NaN     1.0      NaN
2019-07-21            1.0      2.0     1.0      NaN

In [82]:
episode[episode.stimulus == 'hypertrophy'].pivot_table(index = grouper, columns = ['category', 'hypertrophy_range'], values = 'id', aggfunc = 'count')

category             Back            Chest         
hypertrophy_range [7, 12) [12, 20) [7, 12) [12, 20)
date                                               
2019-06-09            NaN      1.0     1.0      NaN
2019-06-30            3.0      2.0     3.0      2.0
2019-07-07            2.0      2.0     1.0      4.0
2019-07-14            NaN      NaN     2.0      1.0
2019-07-21            1.0      2.0     2.0      NaN

In [83]:
episode[episode.stimulus == 'strength'].pivot_table(index = grouper, columns = ['variant', 'strength_range'], values = 'id', aggfunc = 'count')

variant          pull           push        
strength_range [1, 7) [7, 20) [1, 7) [7, 20)
date                                        
2019-07-07          3       3      4       5
2019-07-14          4       6      3       7
2019-07-21          4       2      2       9

In [84]:
episode[episode.stimulus == 'strength'].groupby([grouper, 'variant', 'strength_range']).id.count().groupby(['date', 'variant']).apply(lambda s: s / s.sum()).round(2).unstack(level = ['variant', 'strength_range'])

variant          pull           push        
strength_range [1, 7) [7, 20) [1, 7) [7, 20)
date                                        
2019-07-07       0.50    0.50   0.44    0.56
2019-07-14       0.40    0.60   0.30    0.70
2019-07-21       0.67    0.33   0.18    0.82

In [85]:
episode[episode.stimulus == 'strength'].pivot_table(index = grouper, columns = ['category', 'strength_range'], values = 'id', aggfunc = 'count')

category         Back          Biceps  Chest            Legs Shoulders Triceps
strength_range [1, 7) [7, 20) [7, 20) [1, 7) [7, 20) [7, 20)   [7, 20) [7, 20)
date                                                                          
2019-06-30        NaN     NaN     2.0    NaN     NaN     NaN       NaN     2.0
2019-07-07        3.0     3.0     2.0    4.0     3.0     NaN       2.0     1.0
2019-07-14        4.0     6.0     2.0    3.0     5.0     1.0       2.0     2.0
2019-07-21        4.0     2.0     2.0    2.0     5.0     2.0       2.0     2.0

In [86]:
episode[episode.stimulus == 'strength'].groupby([grouper, 'category', 'strength_range']).id.count().groupby(['date', 'category']).apply(lambda s: s / s.sum()).round(2).unstack(level = ['category', 'strength_range'])

category        Biceps Triceps   Back          Chest         Shoulders    Legs
strength_range [7, 20) [7, 20) [1, 7) [7, 20) [1, 7) [7, 20)   [7, 20) [7, 20)
date                                                                          
2019-06-30         1.0     1.0    NaN     NaN    NaN     NaN       NaN     NaN
2019-07-07         1.0     1.0   0.50    0.50   0.57    0.43       1.0     NaN
2019-07-14         1.0     1.0   0.40    0.60   0.38    0.62       1.0     1.0
2019-07-21         1.0     1.0   0.67    0.33   0.29    0.71       1.0     1.0

### Stimulating Reps

5 - RIR = number of stimulating reps

5 - (10 - RPE) = n_stim reps

RPE - 5 = n_stim reps

https://medium.com/@SandCResearch/what-is-training-volume-286b8da6f427

In [88]:
episode.pivot_table(index = grouper, columns  = 'variant', values = 'stim_reps', aggfunc = sum)

variant,pull,push
date,,
2019-06-09,3.0,4.0
2019-06-30,12.0,20.5
2019-07-07,31.0,32.0
2019-07-14,26.5,35.5
2019-07-21,36.0,36.5


In [89]:
episode.pivot_table(index = grouper, columns  = 'category', values = 'stim_reps', aggfunc = sum)

category,Back,Biceps,Chest,Legs,Shoulders,Triceps
date,,,,,,
2019-06-09,3.0,NaN,4.0,NaN,NaN,NaN
2019-06-30,12.0,3.0,20.5,NaN,NaN,6.0
2019-07-07,31.0,9.0,35.5,NaN,4.5,5.0
2019-07-14,26.5,5.5,37.5,1.0,8.0,8.0
2019-07-21,36.0,10.0,30.5,4.0,7.0,6.0


## Load

In [90]:
episode.pivot_table(index = grouper, columns = 'variant', values = 'intensity', aggfunc = 'mean').round(2)

variant,pull,push
date,,
2019-06-09,0.69,0.67
2019-06-30,0.66,0.68
2019-07-07,0.74,0.75
2019-07-14,0.78,0.75
2019-07-21,0.76,0.74


In [91]:
episode.pivot_table(index = grouper, columns = 'category', values = 'intensity', aggfunc = 'mean').round(2)

category,Back,Biceps,Chest,Legs,Shoulders,Triceps
date,,,,,,
2019-06-09,0.69,NaN,0.67,NaN,NaN,NaN
2019-06-30,0.66,0.71,0.68,NaN,NaN,0.70
2019-07-07,0.74,0.71,0.73,NaN,0.79,0.71
2019-07-14,0.78,0.71,0.73,0.79,0.79,0.71
2019-07-21,0.76,0.76,0.72,0.75,0.79,0.71


## Fatigue

In [92]:
episode.pivot_table(index = grouper, columns = 'variant', values = 'rpe', aggfunc = 'mean')

variant,pull,push
date,,
2019-06-09,8.00,9.000000
2019-06-30,7.40,9.100000
2019-07-07,8.10,7.750000
2019-07-14,8.15,8.227273
2019-07-21,9.00,8.458333


In [93]:
episode.pivot_table(index = grouper, columns = 'category', values = 'rpe', aggfunc = 'mean')

category,Back,Biceps,Chest,Legs,Shoulders,Triceps
date,,,,,,
2019-06-09,8.00,NaN,9.000000,NaN,NaN,NaN
2019-06-30,7.40,6.50,9.100000,NaN,NaN,8.0
2019-07-07,8.10,9.50,8.041667,NaN,7.25,10.0
2019-07-14,8.15,7.75,8.409091,6.0,9.00,9.0
2019-07-21,9.00,10.00,8.944444,7.0,8.50,8.0


## Workout Explorer

In [96]:
dates = sorted(episode.date.unique())

In [97]:
wk_weight = episode[episode.date == dates[-1]].pivot(index = 'exercise', columns = 'ex_rep_no', values = 'weight').fillna('').astype(str).apply(lambda s: s.str.replace('.0$', ''))

In [98]:
wk_reps = episode[episode.date == dates[-1]].pivot(index = 'exercise', columns = 'ex_rep_no', values = 'reps').fillna('').astype(str).apply(lambda s: s.str.replace('.0$', ''))

In [99]:
wk_rpe = episode[episode.date == dates[-1]].pivot(index = 'exercise', columns = 'ex_rep_no', values = 'rpe').fillna('').astype(str).apply(lambda s: s.str.replace('.0$', ''))

In [100]:
wk_weight

ex_rep_no,0,1,2
exercise,,,
Close Grip Dip,209,209,
Flat Barbell Bench Press,185,185,185
Incline Barbell Bench Press,125,125,
Incline Dumbbell Hammer Curl,35,35,
Pull Up,209,209,
Seated Cable Row,210,210,210


In [101]:
wk_reps

ex_rep_no,0,1,2
exercise,,,
Close Grip Dip,12,12,
Flat Barbell Bench Press,10,10,10
Incline Barbell Bench Press,12,12,
Incline Dumbbell Hammer Curl,9,8,
Pull Up,12,12,
Seated Cable Row,9,7,6


In [102]:
wk_rpe

ex_rep_no,0,1,2
exercise,,,
Close Grip Dip,7,9,
Flat Barbell Bench Press,8,9,9.5
Incline Barbell Bench Press,9,10,
Incline Dumbbell Hammer Curl,10,10,
Pull Up,9,10,
Seated Cable Row,10,10,10


In [103]:
wk_weight.astype(str) + 'x' + wk_reps.astype(str) + 'x' + wk_rpe.astype(str)

ex_rep_no,0,1,2
exercise,,,
Close Grip Dip,209x12x7,209x12x9,xx
Flat Barbell Bench Press,185x10x8,185x10x9,185x10x9.5
Incline Barbell Bench Press,125x12x9,125x12x10,xx
Incline Dumbbell Hammer Curl,35x9x10,35x8x10,xx
Pull Up,209x12x9,209x12x10,xx
Seated Cable Row,210x9x10,210x7x10,210x6x10
